# -1. Setting

If you run in jupyter, turn

```
colab = False
```




In [1]:
colab = True
if colab:
    !pip install swig
    !pip install gym==0.21 pyvirtualdisplay > /dev/null 2>&1
    !apt-get install -y xvfb python-opengl ffmpeg > /dev/null 2>&1
    !apt-get update > /dev/null 2>&1
    !apt-get install cmake > /dev/null 2>&1
    !pip install --upgrade setuptools 2>&1
    !pip install ez_setup > /dev/null 2>&1
    !pip3 install box2d-py
    !pip3 install gym[Box_2D]
    !pip install pygame
    !pip install pyglet==1.5.27

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 36.0 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 804.0/804.0 kB 50.3 MB/s eta 0:00:00
  Attempting uninstall: setuptools
    Found existing installation: setuptools 67.7.2
    Uninstalling setuptools-67.7.2:
      Successfully uninstalled setuptools-67.7.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython 7.34.0 requires jedi>=0.16, which is not installed.


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.5/374.5 kB 32.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for box2d-py: filename=box2d_py-2.3.8-cp310-cp310-linux_x86_64.whl size=2811826 sha256=181c4effef4912a8f0b52c4d6f7981a29ff452d47393b28a2adb2630262b03ac
  Stored in directory: /root/.cache/pip/wheels/47/01/d2/6a780da77ccb98b1d2facdd520a8d10838a03b590f6f8d50c0
Successfully built box2d-py
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 59.9 MB/s eta 0:00:00


In [2]:
colab=True
if colab:
    from google.colab import drive
    drive.mount('/content/drive')

    %cd /content/drive/MyDrive/Colab Notebooks/양인순 교수님/day3/ddpg
    !ls

Mounted at /content/drive
/content/drive/MyDrive/Colab Notebooks/양인순 교수님/day3/ddpg
buffer.py		       ddpg.ipynb   snapshots	    utils.py
chap5_ddpg_samsung_merged.pdf  __pycache__  snapshot_saved  video


In [3]:
import torch
import torch.nn as nn
from torch.nn import MSELoss
import torch.nn.functional as F
import copy
import os
import numpy as np
from tqdm import tqdm
import torch
from torch.optim import Adam
#from buffer import ReplayBuffer
from utils import save_snapshot, recover_snapshot, load_model
import gym

In [4]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('current device =', device)

current device = cuda


# 0. Define Q-network & policy-network

In [54]:
# critic network definition
# multi-layer perceptron (with 2 hidden layers)
class Critic(nn.Module):
    def __init__(self, obs_dim, act_dim, hidden1, hidden2):
        super(Critic, self).__init__()
        self.fc1 = nn.Linear(obs_dim + act_dim, hidden1)
        self.fc2 = nn.Linear(hidden1, hidden2)

        self.fc3 = nn.Linear(hidden2, 1)


    def forward(self, obs, act):
        x = torch.cat([obs, act], dim=1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))

        return self.fc3(x)


# actor network definition
# multi-layer perceptron (with 2 hidden layers)
class Actor(nn.Module):
    def __init__(self, obs_dim, act_dim, ctrl_range, hidden1, hidden2):
        super(Actor, self).__init__()
        self.fc1 = nn.Linear(obs_dim, hidden1)
        self.fc2 = nn.Linear(hidden1, hidden2)
        self.fc3 = nn.Linear(hidden2, act_dim)
        self.ctrl_range = ctrl_range

    def forward(self, obs):
        x = F.relu(self.fc1(obs))
        x = F.relu(self.fc2(x))

        return self.ctrl_range * torch.tanh(self.fc3(x))

# 1. Define DDPG agent

In [55]:
class DDPGAgent:
    def __init__(self, obs_dim, act_dim, ctrl_range, hidden1, hidden2):

        # networks
        self.actor = Actor(obs_dim, act_dim, ctrl_range, hidden1, hidden2).to(device)
        self.critic = Critic(obs_dim, act_dim, hidden1, hidden2).to(device)

    def act(self, obs):
        # numpy ndarray to torch tensor
        # we first add an extra dimension
        obs = obs[np.newaxis, ...]
        with torch.no_grad():
            obs_tensor = torch.Tensor(obs).to(device)
            # TODO : get agent action from policy network (self.actor)
            act_tensor = self.actor(obs_tensor)

        # torch tensor to numpy ndarray
        # remove extra dimension
        action = act_tensor.cpu().detach().numpy()
        action = np.squeeze(action, axis=0)

        return action


## 1.1.Test

In [56]:
agent = DDPGAgent(4, 2, 3, 32, 32)
action = agent.act(np.array([3., -1., 2., -5.]))
print(action)

[-0.4712096   0.09887823]


# 2. Implement one-step param update

Actor update? \\
First Observe
\begin{equation*}
\nabla_{\phi} Q_{\theta}(s_t, \mu_{\phi}(s_t)) =  \nabla_{\phi} \mu_{\phi}(s_t) \cdot \nabla_{a} Q(s_t, a)|_{a={\mu_{\phi}(s)}}.
\end{equation*}
Thus, we have
\begin{equation*}
\nabla_\phi J(\phi) \approx \frac{1}{N}\sum_{i = 1}^N \nabla_{\phi} \mu_{\phi}(s_t) \cdot \nabla_{a} Q(s_t, a)|_{a={\mu_{\phi}(s)}}  = \nabla_\phi\left( \frac{1}{N}\sum_{i = 1}^N Q_{\theta}(s_t, \mu_{\phi}(s_t)) \right).
\end{equation*}

In [57]:
import numpy
from collections import namedtuple
Batch = namedtuple( 'Batch' , ['obs', 'act', 'rew', 'next_obs', 'done'])

class ReplayBuffer:
    def __init__(self, obs_dim, act_dim, limit):
      self.states= Memory( shape=(obs_dim,), limit = limit )
      self.actions= Memory( shape=(act_dim,), limit = limit )
      self.rewards= Memory( shape=(1,), limit = limit )
      self.next_states= Memory( shape=(obs_dim,), limit = limit )
      self.terminals= Memory( shape=(1,), limit = limit )
      self.limit=limit
      self.size=0

    def append(self, obs, action, next_obs, rew, done):
      self.states.append(obs)
      self.actions.append(action)
      self.next_states.append(next_obs)
      self.rewards.append(rew)
      self.terminals.append(done)
      # self.size = len(self.states)
      # self.size = len(self.states.size)
      self.size = self.states.size

    def sample_batch(self, batch_size):
      rng = np.random.default_rng()
      # idxs = rnf.choice(self.size, batch_size)
      idxs = rng.choice(self.size, batch_size)

      states = self.states.get_batch(idxs)
      actions = self.actions.get_batch(idxs)
      rewards = self.rewards.get_batch(idxs)
      obs = self.states.get_batch(idxs)
      next_obs = self.next_states.get_batch(idxs)
      dones= self.terminals.get_batch(idxs)

      batch = Batch(obs=states, act=actions, rew=rewards, next_obs=next_obs, done=dones)

      return batch

class Memory:

  def __init__(self, shape, limit):

    self.start=0
    self.limit = limit
    self.size = 0 #memory
    self.shape = shape
    self.data = np.zeros((self.limit,) + shape )

  def append(self, data):
    if self.size < self.limit:
      self.size += 1
    else:
      self.start = (self.start + 1) % self.limit
    self.data[(self.start + self.size - 1) % (self.limit)] = data


  def get_batch(self, idxs):
    return self.data[(self.start + idxs) % self.limit]



In [58]:
#batch.obs, batch.action

In [59]:
def update(agent, replay_buf, gamma, actor_optim, critic_optim, target_actor, target_critic, tau, batch_size):
    # agent : agent with networks to be trained
    # replay_buf : replay buf from which we sample a batch
    # actor_optim / critic_optim : torch optimizers
    # tau : parameter for soft target update

    batch = replay_buf.sample_batch(batch_size=batch_size)

    # target construction does not need backward ftns
    with torch.no_grad():
        # unroll batch
        obs = torch.Tensor(batch.obs).to(device)
        act = torch.Tensor(batch.act).to(device)
        next_obs = torch.Tensor(batch.next_obs).to(device)
        rew = torch.Tensor(batch.rew).to(device)
        done = torch.Tensor(batch.done).to(device)

        ################
        # train critic #
        ################
        mask = 1. - done
        target = rew + gamma * mask * target_critic(next_obs, target_actor(next_obs))
    #r + gamma v(s') - v(s)
    out = agent.critic(obs, act)

    # TODO : Build critic MSELoss by yourself!
    # Hint : use torch.mean
    # critic_loss = torch.mean((out-target)**2)
    loss_ftn = MSELoss()
    critic_loss = loss_ftn(out, target)


    critic_optim.zero_grad()
    critic_loss.backward()
    critic_optim.step()

    ###############
    # train actor #
    ###############

    # freeze critic during actor training (why?)
    for p in agent.critic.parameters():
        p.requires_grad_(False)

    # TODO : actor loss construction! (Warning: sign of the loss?)
    actor_loss = -torch.mean(agent.critic(obs, agent.actor(obs)))


    actor_optim.zero_grad()
    actor_loss.backward()
    actor_optim.step()


    # unfreeze critic after actor training
    for p in agent.critic.parameters():
        p.requires_grad_(True)

    # soft target update (both actor & critic network)
    for p, targ_p in zip(agent.actor.parameters(), target_actor.parameters()):
        targ_p.data.copy_((1. - tau) * targ_p + tau * p)
    for p, targ_p in zip(agent.critic.parameters(), target_critic.parameters()):
        targ_p.data.copy_((1. - tau) * targ_p + tau * p)


    return

In [60]:
def evaluate(agent, env, num_episodes=5):

    sum_scores = 0.

    for i in range(num_episodes):
        obs = env.reset()
        done = False
        score = 0.

        while not done:
            action = agent.act(obs)
            obs, rew, done, _ = env.step(action)
            score += rew
        sum_scores += score
    avg_score = sum_scores / num_episodes

    return avg_score

# 3. Combining these, we finally have...

In [61]:
def train(agent, env, gamma,
          actor_lr, critic_lr, tau, noise_std,
          ep_len, num_updates, batch_size,
          init_buffer=5000, buffer_size=100000,
          start_train=2000, train_interval=50,
          eval_interval=2000, snapshot_interval=10000, path=None):

    target_actor = copy.deepcopy(agent.actor)
    target_critic = copy.deepcopy(agent.critic)

    # freeze target networks
    for p in target_actor.parameters():
        p.requires_grad_(False)
    for p in target_critic.parameters():
        p.requires_grad_(False)

    actor_optim = Adam(agent.actor.parameters(), lr=actor_lr)
    critic_optim = Adam(agent.critic.parameters(), lr=critic_lr)

    if path is not None:
        recover_snapshot(path, agent.actor, agent.critic,
                   target_actor, target_critic,
                   actor_optim, critic_optim,
                   device=device
                  )
        # load snapshot

    obs_dim = env.observation_space.shape[0]
    act_dim = env.action_space.shape[0]
    ctrl_range = env.action_space.high[0]

    replay_buf = ReplayBuffer(obs_dim, act_dim, buffer_size)

    save_path = './snapshots/'
    os.makedirs(save_path, exist_ok=True)

    test_env = copy.deepcopy(env)

    # main loop
    obs = env.reset()
    done = False
    step_count = 0
    ep = 0
    for t in range(num_updates + 1):
        if t < init_buffer:
            # perform random action until we collect sufficiently many samples
            # this is for exploration purpose
            action = env.action_space.sample()
        else:
            # executes noisy action
            # a_t = \pi(s_t) + N(0, \sigma^2)
            action = agent.act(obs) + noise_std * np.random.randn(act_dim)
            action = np.clip(action, -ctrl_range, ctrl_range)

        next_obs, rew, done, _ = env.step(action)
        step_count += 1
        if step_count == ep_len:
            # if the next_state is not terminal but done is set to True by gym env wrapper
            done = False

        replay_buf.append(obs, action, next_obs, rew, done)
        obs = next_obs

        if done == True or step_count == ep_len:
            # reset environment if current environment reaches a terminal state
            # or step count reaches predefined length
            obs = env.reset()
            done = False
            step_count = 0
            ep += 1

        if t % eval_interval == 0:
            avg_score = evaluate(agent, test_env, num_episodes=5)
            print('[iter {} / ep {}] average score = {:.4f} (over 5 episodes)'.format(t, ep, avg_score))

        if t > start_train and t % train_interval == 0:
            # start training after fixed number of steps
            # this may mitigate overfitting of networks to the
            # small number of samples collected during the initial stage of training
            for _ in range(train_interval):
                update(agent,
                       replay_buf,
                       gamma,
                       actor_optim,
                       critic_optim,
                       target_actor,
                       target_critic,
                       tau,
                       batch_size
                      )
        if t % snapshot_interval == 0:
            snapshot_path = save_path + 'iter{}_'.format(t)
            # save weight & training progress
            save_snapshot(snapshot_path, agent.actor, agent.critic,
                          target_actor, target_critic,
                          actor_optim, critic_optim)

# 4. Let's test the code!

In [62]:
import gym

In [63]:
env = gym.make('LunarLanderContinuous-v2')
obs_dim = env.observation_space.shape[0]
act_dim = env.action_space.shape[0]
ctrl_range = env.action_space.high[0]

print('observation space dim : {} / action space dim : {}'.format(obs_dim, act_dim))
print('ctrl range : ', ctrl_range)

observation space dim : 8 / action space dim : 2
ctrl range :  1.0


/usr/local/lib/python3.10/dist-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(


In [64]:
agent = DDPGAgent(obs_dim=obs_dim, act_dim=act_dim, ctrl_range=ctrl_range, hidden1=256, hidden2=256)

In [65]:
gamma = 0.99
actor_lr = 1e-4
critic_lr = 1e-3
tau = 1e-3
noise_std = 0.1
ep_len = 1000
num_updates = 500000
batch_size = 128

In [66]:
train(agent, env, gamma,
      actor_lr, critic_lr, tau, noise_std,
      ep_len, num_updates, batch_size,
      init_buffer=5000, buffer_size=100000,
      start_train=2000, train_interval=50,
      eval_interval=5000
     )

[iter 0 / ep 0] average score = -340.0386 (over 5 episodes)
[iter 5000 / ep 47] average score = -148.6363 (over 5 episodes)
[iter 10000 / ep 62] average score = -160.3952 (over 5 episodes)
[iter 15000 / ep 75] average score = -139.7056 (over 5 episodes)


KeyboardInterrupt: ignored

# 5. Watch the trained agent!

In [67]:
ITER = 0
path = './snapshots/iter'+str(ITER)+'_model.pth.tar'
actor_optim = Adam(agent.actor.parameters(), lr=actor_lr)
critic_optim = Adam(agent.critic.parameters(), lr=critic_lr)


In [68]:
!pip install pyvirtualdisplay

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [69]:
import gym
from gym.wrappers.record_video import RecordVideo
import glob
import io
import base64
from IPython.display import HTML
from pyvirtualdisplay import Display
from IPython import display as ipythondisplay

display = Display(visible=0, size=(1400, 900))
display.start()

"""
Utility functions to enable video recording of gym environment
and displaying it.
To enable video, just do "env = wrap_env(env)""
"""
def show_video(env_id):
    mp4list = glob.glob('video/'+env_id+'/*.mp4')
    if len(mp4list) > 0:
      mp4 = mp4list[0]
      video = io.open(mp4, 'r+b').read()
      encoded = base64.b64encode(video)
      ipythondisplay.display(HTML(data='''<video alt="test" autoplay
                  loop controls style="height: 400px;">
                  <source src="data:video/mp4;base64,{0}" type="video/mp4" />
              </video>'''.format(encoded.decode('ascii'))))
    else:
        print("Could not find video")


def wrap_env(env, env_id):
#    env = Monitor(env, './video', force=True)
    env=RecordVideo(env, './video/'+env_id,  episode_trigger = lambda episode_number: True)
    return env

In [70]:

env = gym.make('LunarLanderContinuous-v2')
env_id = 'LunarLanderContinuous-v2'
if colab:
  env = wrap_env(env,env_id)
obs = env.reset()
done = False
score = 0.

ITER = 0
agent = DDPGAgent(obs_dim=obs_dim, act_dim=act_dim, ctrl_range=ctrl_range, hidden1=256, hidden2=256)
recover_snapshot(path, agent.actor, agent.critic,
            copy.deepcopy(agent.actor), copy.deepcopy(agent.critic),
            actor_optim, critic_optim,
            device=device
          )

while not done:
    env.render(mode='human')
    obs, rew, done, _ = env.step(agent.act(obs))
    score += rew
print('score : ', score)
env.close()

if colab:
    show_video(env_id)

/usr/local/lib/python3.10/dist-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/wrappers/record_video.py:78: UserWarning: WARN: Overwriting existing videos at /content/drive/MyDrive/Colab Notebooks/양인순 교수님/day3/ddpg/video/LunarLanderContinuous-v2 folder (try specifying a different `video_folder` for the `RecordVideo` wrapper if this is not desired)
  logger.warn(
/usr/local/lib/python3.10/dist-packages/gym/wrappers/monit

recovering snapshot...


/usr/local/lib/python3.10/dist-packages/gym/core.py:43: DeprecationWarning: WARN: The argument mode in render method is deprecated; use render_mode during environment initialization instead.
See here for more information: https://www.gymlibrary.ml/content/api/
  deprecation(


score :  -338.4466693487299
